In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/renfe-original/renfe.csv
/kaggle/input/spanish-high-speed-rail-system-ticket-pricing/renfe.csv


# **CS334 Project -- Preprocessing**
## —— George Saito, Qiao Lin

## 1. Import Dataset

In [2]:
renfe = pd.read_csv("../input/spanish-high-speed-rail-system-ticket-pricing/renfe.csv")


## 2. Explore the data

In [3]:
print(renfe.shape)

(7671354, 9)


In [4]:
renfe = renfe.head(120000)

### 3. Preprocessing

* **Convert the datetime object.**

In [5]:
for i in ['insert_date','start_date','end_date']:
    renfe[i] = pd.to_datetime(renfe[i])

* **Check and drop the null values.** 

In [6]:
# Check for null value
renfe.isnull().mean()*100

insert_date     0.000000
origin          0.000000
destination     0.000000
start_date      0.000000
end_date        0.000000
train_type      0.000000
price          58.585000
train_class     0.356667
fare            0.356667
dtype: float64

In [7]:
print(renfe.info)
renfe.isnull().any()
renfe = renfe.dropna()

<bound method DataFrame.info of                insert_date  origin destination          start_date  \
0      2019-04-11 21:49:46  MADRID   BARCELONA 2019-04-18 05:50:00   
1      2019-04-11 21:49:46  MADRID   BARCELONA 2019-04-18 06:30:00   
2      2019-04-11 21:49:46  MADRID   BARCELONA 2019-04-18 07:00:00   
3      2019-04-11 21:49:46  MADRID   BARCELONA 2019-04-18 07:30:00   
4      2019-04-11 21:49:46  MADRID   BARCELONA 2019-04-18 08:00:00   
...                    ...     ...         ...                 ...   
119995 2019-04-13 01:27:06  MADRID    VALENCIA 2019-06-11 06:08:00   
119996 2019-04-13 01:27:06  MADRID    VALENCIA 2019-06-11 12:21:00   
119997 2019-04-13 01:27:10  MADRID    VALENCIA 2019-05-12 18:40:00   
119998 2019-04-13 01:27:10  MADRID    VALENCIA 2019-05-12 10:40:00   
119999 2019-04-13 01:27:10  MADRID    VALENCIA 2019-05-12 11:10:00   

                  end_date train_type   price   train_class        fare  
0      2019-04-18 08:55:00        AVE   68.95    Pref

* **Convert 'origin' and 'destination' to 'route' feature. **

In [8]:
# Changing the origin and destination to the routes
renfe['route'] = renfe['origin'] + ' to ' + renfe['destination']

* **Feature extraction**
    + We seperate the previously converted feature "datetime" into several fetures like "month", "day", and "time", since datetime itself is a complex feature. 

In [9]:
# Extracting features from 'start_date' and 'end_date'
renfe['month'] = renfe['start_date'].apply(lambda d:d.month)
renfe['day_name'] = renfe['start_date'].apply(lambda d: d.weekday_name)
renfe['quarter'] = renfe['start_date'].apply(lambda d: d.quarter)
renfe['travel_time'] = (renfe['end_date']-renfe['start_date'])/np.timedelta64(1, 'm')
renfe['start_hour'] = renfe['start_date'].apply(lambda d: d.hour)
renfe['end_hour'] = renfe['end_date'].apply(lambda d: d.hour)
print(renfe.info)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: `weekday_name` is deprecated and will be removed in a future version. Use `day_name` instead
  This is separate from the ipykernel package so we can avoid doing imports until


<bound method DataFrame.info of               insert_date  origin destination          start_date  \
0     2019-04-11 21:49:46  MADRID   BARCELONA 2019-04-18 05:50:00   
1     2019-04-11 21:49:46  MADRID   BARCELONA 2019-04-18 06:30:00   
2     2019-04-11 21:49:46  MADRID   BARCELONA 2019-04-18 07:00:00   
3     2019-04-11 21:49:46  MADRID   BARCELONA 2019-04-18 07:30:00   
4     2019-04-11 21:49:46  MADRID   BARCELONA 2019-04-18 08:00:00   
...                   ...     ...         ...                 ...   
56063 2019-04-12 10:27:43  MADRID    VALENCIA 2019-06-06 16:05:00   
56064 2019-04-12 10:27:43  MADRID    VALENCIA 2019-06-06 12:21:00   
56065 2019-04-12 10:33:05  MADRID    VALENCIA 2019-06-03 12:21:00   
56066 2019-04-12 10:33:05  MADRID    VALENCIA 2019-06-03 06:08:00   
56067 2019-04-12 10:33:05  MADRID    VALENCIA 2019-06-03 16:05:00   

                 end_date train_type   price   train_class        fare  \
0     2019-04-18 08:55:00        AVE   68.95    Preferente       

* **Deleting transformed features**

In [10]:
print(renfe.columns)
renfe.drop(['origin', 'destination', 'insert_date', 'start_date', 'end_date'], axis=1, inplace=True)
print(renfe.columns)

Index(['insert_date', 'origin', 'destination', 'start_date', 'end_date',
       'train_type', 'price', 'train_class', 'fare', 'route', 'month',
       'day_name', 'quarter', 'travel_time', 'start_hour', 'end_hour'],
      dtype='object')
Index(['train_type', 'price', 'train_class', 'fare', 'route', 'month',
       'day_name', 'quarter', 'travel_time', 'start_hour', 'end_hour'],
      dtype='object')


* **One-Hot Encoding**

In [11]:
x_categ = renfe[['train_type', 'train_class', 'fare', 'day_name', 'quarter', 'route']]
x_oh = renfe[['price', 'month', 'travel_time', 'start_hour', 'end_hour']]
oh = preprocessing.OneHotEncoder()
oh_categ = oh.fit_transform(x_categ)
oh_feat = oh.get_feature_names(['train_type', 'train_class', 'fare', 'day_name', 'quarter', 'route'])
print(len(oh_feat))
print(len(np.transpose(oh_categ.toarray())))
i = 0
for c in oh_feat:
    x_oh[c] = np.transpose(oh_categ.toarray())[i]
    i += 1
print(x_oh)

29
29
        price  month  travel_time  start_hour  end_hour  train_type_ALVIA  \
0       68.95      4        185.0           5         8               0.0   
1       75.40      4        170.0           6         9               0.0   
2      106.75      4        150.0           7         9               0.0   
3       90.50      4        190.0           7        10               0.0   
4       88.95      4        150.0           8        10               0.0   
...       ...    ...          ...         ...       ...               ...   
56063   28.35      6        402.0          16        22               0.0   
56064   28.35      6        403.0          12        19               0.0   
56065   28.35      6        403.0          12        19               0.0   
56066   28.35      6        407.0           6        12               0.0   
56067   28.35      6        402.0          16        22               0.0   

       train_type_AV City  train_type_AVE  train_type_AVE-LD  \
0    

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


* **Extract y-label and split the dataset to train and test datasets**

In [12]:
#from sklearn.model_selection import train_test_split
y = x_oh['price']
xTrain_oh, xTest_oh, yTrain_oh, yTest_oh = train_test_split(x_oh.drop(columns=['price']), y, test_size=0.3, random_state=334)

### 4. Output the dataset as csv file

In [13]:
xTrain_oh.to_csv("xTrain_renfe_oh.csv", index=False)
xTest_oh.to_csv("xTest_renfe_oh.csv", index=False)
yTrain_oh.to_csv("yTrain_renfe_oh.csv", header='label', index=False)
yTest_oh.to_csv("yTest_renfe_oh.csv", header='label', index=False)